In [1]:
import numpy as np
from fastai import *
from fastai.vision import *
import torch
import os
from torchvision import transforms

**data from https://www.kaggle.com/mloey1/ahcd1**

In [2]:
data_path = Path('data')

In [9]:
test_x = np.genfromtxt(data_path/'csvTestImages 3360x1024.csv', delimiter=',')
test_y = np.genfromtxt(data_path/'csvTestLabel 3360x1.csv', delimiter=',')
test_y = test_y.astype(int) - 1
test_x = test_x.astype(float)


test_x.shape, test_y.shape, test_x[0].shape, test_y.max(), test_y.min(), max(test_x.max(axis=1))

((3360, 1024), (3360,), (1024,), 27, 0, 255.0)

In [27]:
class ArabicMnist(Dataset):
    def __init__(self, X, y=None):
        super().__init__()
        self.classes = np.unique(y)
        self.c = len(np.unique(y))
        self.X = X
        if y is not None: self.y = y

    def __getitem__(self, i):
        image = self.X[i]
        arr = (image - image.mean()) / image.std() # normalizes between -1 and +1
        arr = (arr + 1) / 2 * 255 # moves it between 0 and 255
#         arr = np.clip(arr, 0, 255).astype(np.uint8),
        arr3D = np.tile(arr[...,None],3)
#         arr3D = 256 - arr3D
        arr3D = np.transpose(arr3D, (2, 1, 0))
        new_tensor = torch.from_numpy(arr3D).float()
        return (new_tensor, self.y[i])
    
    def __len__(self): return len(self.X)

In [28]:
test_x_reshaped = test_x.reshape([-1, 32, 32])

In [29]:
test_x_reshaped.shape, test_y.shape

((3360, 32, 32), (3360,))

In [30]:
AM = ArabicMnist(test_x_reshaped[:2720], test_y[:2720])
AMV = ArabicMnist(test_x_reshaped[2720:], test_y[2720:])

In [31]:
data = DataBunch.create(AM, AMV, bs=8)


In [32]:
learn = create_cnn(data, models.resnet18, metrics=error_rate)

In [33]:
# %%debug
learn.fit(10, 1e-9)

Total time: 01:38
epoch  train_loss  valid_loss  error_rate
1      0.014538    -0.041725   0.964063    (00:09)
2      -0.040858   -0.014340   0.956250    (00:09)
3      -0.091720   -0.062538   0.968750    (00:09)
4      0.009249    -0.020415   0.967188    (00:09)
5      0.043340    0.002689    0.962500    (00:09)
6      0.051320    -0.042611   0.971875    (00:09)
7      -0.065322   -0.011302   0.973437    (00:09)
8      0.009185    0.012544    0.973437    (00:09)
9      0.044579    -0.020431   0.970312    (00:09)
10     0.015424    0.010244    0.973437    (00:09)

